In [1]:
#Imports
import nltk
import os
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle
from tensorflow.python.framework import ops

#Loading Data
with open("intents.json") as file:
	data = json.load(file)
  
#Initializing empty lists
words = []
labels = []
docs_x = []
docs_y = []

#Looping through our data
for intent in data['intents']:
	for pattern in intent['patterns']:
		pattern = pattern.lower()
    		#Creating a list of words
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent['tag'])

	if intent['tag'] not in labels:
	  labels.append(intent['tag'])
  
stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
	bag = []
	wrds = [stemmer.stem(w) for w in doc]
	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1
	training.append(bag)
	output.append(output_row)
#Converting training data into NumPy arrays
training = np.array(training)
output = np.array(output)

#Saving data to disk
with open("data.pickle","wb") as f:
	pickle.dump((words, labels, training, output),f)
 

ops.reset_default_graph()

net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
x=model.fit(training, output, n_epoch = 200, batch_size = 8, show_metric = True)
model.save("model.tflearn")

Training Step: 799  | total loss: 0.13647 | time: 0.008s
| Adam | epoch: 200 | loss: 0.13647 - acc: 0.9936 -- iter: 24/29
Training Step: 800  | total loss: 0.12704 | time: 0.008s
| Adam | epoch: 200 | loss: 0.12704 - acc: 0.9942 -- iter: 29/29
--
INFO:tensorflow:C:\Users\suriy\apii\app\app\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\suriy\apii\app\app\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\suriy\apii\app\app\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\suriy\apii\app\app\model.tflearn.meta
INFO:tensorflow:100
